# CS 5785 - Final
## Applied Machine Learning

Arpit Sheth *(as3668)*

Eric Nguyen *(en274)*

Samantha Yip *(sty5)*

In [2]:
import setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, Markdown
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer
from autocorrect import spell
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import svm
from sklearn import preprocessing
import csv
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors as KNN

KeyboardInterrupt: 

In [ ]:
TRAIN = 0
TEST = 1
N_TRAIN = 10000
N_TEST = 2000

# These become Pandas DataFrames
descriptions_train = []
descriptions_test = []
features_train = []
features_test = []
ifeatures_train = []
ifeatures_test = []
tags_train = []
tags_test = []

In [ ]:
setup.get_image(1, TRAIN)

### Get Descriptions and create bag of words

In [ ]:
descriptions_train = setup.get_descriptions(10000,TRAIN)

In [ ]:
descriptions_test = setup.get_descriptions(2000,TEST)

In [ ]:
stop = stopwords.words('english')
plural = WordNetLemmatizer()
for j in range(5):
    descriptions_train['s'+str(j+1)] = descriptions_train['s'+str(j+1)].str.lower()
    descriptions_train['s'+str(j+1)] = descriptions_train['s'+str(j+1)].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    descriptions_train['s'+str(j+1)] = descriptions_train['s'+str(j+1)].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
    for i, sentence in enumerate(descriptions_train['s'+str(j+1)]):
        temp = []
        for word in sentence.split(" "):
            word2 = spell(word)
            word2 = plural.lemmatize(word2,'n')
            word2 = plural.lemmatize(word2,'v')
            word2 = plural.lemmatize(word2,'a')
            word2 = plural.lemmatize(word2,'r')
            temp.append(word2)
        descriptions_train['s'+str(j+1)][i] = ' '.join(temp)

In [ ]:
stop = stopwords.words('english')
plural = WordNetLemmatizer()
for j in range(5):
    descriptions_test['s'+str(j+1)] = descriptions_test['s'+str(j+1)].str.lower()
    descriptions_test['s'+str(j+1)] = descriptions_test['s'+str(j+1)].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    descriptions_test['s'+str(j+1)] = descriptions_test['s'+str(j+1)].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
    for i, sentence in enumerate(descriptions_test['s'+str(j+1)]):
        temp = []
        for word in sentence.split(" "):
            word2 = spell(word)
            word2 = plural.lemmatize(word2,'n')
            word2 = plural.lemmatize(word2,'v')
            word2 = plural.lemmatize(word2,'a')
            word2 = plural.lemmatize(word2,'r')
            temp.append(word2)
        descriptions_test['s'+str(j+1)][i] = ' '.join(temp)

In [ ]:
documents_train = []
for index, row in descriptions_train.iterrows():
    description = ' '.join(row.tolist())
    documents_train.append(description)
documents_train = pd.DataFrame(documents_train, columns=["sentences"])

In [ ]:
documents_test = []
for index, row in descriptions_test.iterrows():
    description = ' '.join(row.tolist())
    documents_test.append(description)
documents_test = pd.DataFrame(documents_test, columns=["sentences"])

In [ ]:
bag_of_words = []
for i in range(10000):
    for j in range(5):
        for word in descriptions_train.get_value(i,"s"+str(j+1)).split(" "):
            bag_of_words.append(word)

In [ ]:
bag_of_words = list(set(bag_of_words))

In [ ]:
bag_of_words_test = []
for sentence in documents_test["sentences"]:
    for word in sentence.split(" "):
        bag_of_words_test.append(word)
bag_of_words_test = list(set(bag_of_words_test))

### Get the full list of subcategories

In [ ]:
tags_train = setup.get_tags(10000,TRAIN)

In [ ]:
subcat_list = []
final_tags_train = []
for i, tags in tags_train.iterrows():
    for tag in tags:
        temp = ''
        for j in tag:
            word = j[1].replace(" ", "")
            subcat_list.append(word)
            temp = temp + ' ' + word
        final_tags_train.append(temp)
        
subcat_list = list(set(subcat_list))

### Conduct TF-IDF

In [ ]:
cv = CountVectorizer(min_df = 3)
X_train_bow = cv.fit_transform(documents_train["sentences"]).toarray()
vocab = np.array(cv.get_feature_names())
transformer = TfidfTransformer()
X_train_tfidf = transformer.fit_transform(X_train_bow).toarray()

# Convert into DataFrames
X_train_tfidf = pd.DataFrame(X_train_tfidf, columns = vocab)
X_train_bow = pd.DataFrame(X_train_bow, columns = vocab)
X_train_tfidf.head()

In [ ]:
cv = CountVectorizer(vocabulary = vocab)
X_test_bow = cv.fit_transform(documents_test["sentences"]).toarray()
transformer = TfidfTransformer()
X_test_tfidf = transformer.fit_transform(X_test_bow).toarray()

# Convert into DataFrames
X_test_tfidf = pd.DataFrame(X_test_tfidf, columns = vocab)

In [ ]:
cv = CountVectorizer(vocabulary = subcat_list)
tags_train_0_1 = cv.fit_transform(final_tags_train).toarray()

### Random Forest Training Model

In [ ]:
predict_tags = []
for i in range(80):
    y = tags_train_0_1[:,i]
    clf = RandomForestRegressor()
    clf.fit(X_train_tfidf, y)
    tag = clf.predict(X_test_tfidf_norm)
    predict_tags.append(tag)

In [ ]:
predict_tags_trans = predict_tags.T
predict_tags_trans.shape

In [ ]:
tags_test = setup.get_tags(2000,TEST)
final_tags_test = []
for tag in tags_test['tags']:
    temp = ''
    for i in tag:
        word = i[1].replace(" ","")
        temp = temp + ' ' + word
    final_tags_test.append(temp)
cv = CountVectorizer(vocabulary = subcat_list)
tags_test_0_1 = cv.fit_transform(final_tags_test).toarray()
tags_test_0_1.shape

### Knn

In [ ]:
test_labels = [i for i in range(2000)]
knn = KNN(n_neighbors = 20)
knn = knn.fit(tags_test_0_1, test_labels)
predictions = knn.kneighbors(predict_tags_trans, return_distance = False)
# predictions = pd.DataFrame(predictions)

In [ ]:
list = []
for i, row in enumerate(predictions):
    temp = ''
    for j, val in enumerate(row):
        temp = temp + ' ' + (str(val) + ".jpg")
    list.append(temp)
images = pd.DataFrame(list, columns = ["Top_20_Image_IDs"])

In [ ]:
index = []
for i in range(2000):
    index.append(str(i)+ ".txt")
index = pd.DataFrame(index, columns = ["Descritpion_ID"])

In [ ]:
result = pd.concat([index, images], axis=1)
result.to_csv("Logistic_Regression.csv", index = False)